In [1]:
## Importing Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, AncillaRegister, Aer, execute, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
from qiskit.circuit.library import MCMT, RYGate, RXGate, PhaseGate
import numpy as np
np.set_printoptions(threshold=np.inf)
import math
from typing import Union

In [45]:
## Settings
d = 1 # of dimensions
M = 1 # of registers (rough estimate for M is the number of particles in the final state, but M should be set so that p_success can be maximized)
n = 1 # of particles in the initial state
N_abs = 1 # of modes for momenta
m = 1 # mass of particles of projectile


nqubits = N_abs * d + d + 1
N_s = 2 ** (N_abs + 1) # size of the lattice per one dimension
V = N_s ** d # volume
N = N_abs * d + d + 1 # of qubits per one particle (magnitude of momentum + sign + occupation)
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.

In [3]:
I = np.array([[1, 0], [0, 1]])
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, 1]])

In [4]:
#初期状態|0000・・・0>を準備する。
def StateZeros(nqubits):
    State = np.zeros(2**nqubits)
    State[0]=1
    return State

In [29]:
def makeGate(N, nqubit, Gate): # action of Gate to n'th qubit (n = 0...N)
    if nqubit == 0:
        tmp = Gate
    else: tmp = I
    for i in range(N):
        B_tmp = np.array((2 ** (i + 2), 2 ** (i + 2)))
        if nqubit == i + 1:
            B_tmp = np.kron(Gate, tmp)
        else: B_tmp = np.kron(I, tmp)
        tmp = B_tmp
    return tmp

In [ ]:
# makeGate(N, (N_abs + 1) * d, X)
# makeGate(1, 1, H)

In [180]:
# if dagger is True, make an annihilation operator, if False, an creation operator
# ireg means the number of the register acted
# qmom means that the momentum with sign
def annahilationCreationOpPerReg(qmom, ireg, dagger, N, M):
    if qmom > 0:
        qrow = int(qmom - 1/2 + 2 ** (N-1))
    elif qmom < 0:
        qrow = int(np.abs(qmom + 1/2) + 2 ** (N-1) + 2 ** N_abs)
    tmp_op = np.zeros((2 ** N, 2 ** N))
    if dagger == False:
        tmp_op[0][qrow] = 1
    else:
        tmp_op[qrow][0] = 1
    identity = np.eye(2 ** N)
    if ireg == 0:
        tmp = tmp_op
    else: tmp = identity
    # print(tmp)
    for i in range(M-1):
        # print(i)
        B_tmp = np.array((2 ** (i + 2), 2 ** (i + 2)))
        if ireg == i + 1:
            B_tmp = np.kron(tmp_op, tmp)
        else: B_tmp = np.kron(identity, tmp)
        tmp = B_tmp
    return tmp

In [194]:
def annahilationCreationOp(qmom, dagger):
    tmp_op = np.zeros((2 ** (N * M), 2 ** (N * M)))
    # print(tmp_op.size)
    for ireg in range(M):
        # print(annahilationCreationOpPerReg(qmom, ireg, dagger, N, M).size)
        tmp_op += annahilationCreationOpPerReg(qmom, ireg, dagger, N, M)
    return tmp_op * (1/np.sqrt(M))

In [174]:
annahilationCreationOpPerReg(-1, 0, False, 3, 1)
# annahilationCreationOp(1, True)

array([[0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [161]:
def squeezeOpPerMom(qmom):
    zq = (1/2) * np.log(np.abs(qmom + 1/2))
    A = annahilationCreationOp(qmom, True)
    B = annahilationCreationOp(-qmom, True)
    C = annahilationCreationOp(-qmom, False)
    D = annahilationCreationOp(qmom, False)
    ln_sq = -zq * (A @ B - C @ D)
    sq = np.exp(ln_sq)
    return sq

In [187]:
def squeezeOp():
    sq = np.eye(2 ** (N * M))
    for q in range(N_abs + 1):
        print(q)
        # print(squeezeOpPerMom(q+1/2))
        # print(squeezeOpPerMom(-(q+1/2)))
        sq @= squeezeOpPerMom(q + 1/2)
        sq @= squeezeOpPerMom(-(q + 1/2))
    return sq

In [192]:
def phi(n):
    A = annahilationCreationOp(n, True)
    B = annahilationCreationOp(n, False)
    phiGate = (1/np.sqrt(2)) * (A + B)
    return phiGate

In [195]:
def intU(delta, lam):
    ln_UI = 0
    for ipos in range(N_abs * d + d):
        ln_UI += -1j * delta * (lam/24) * phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos)
    uI = np.exp(ln_UI)
    return uI

In [197]:
intU(0.1, 0.1)

UnboundLocalError: local variable 'qrow' referenced before assignment